In [1]:
from mlsquare import imly


In [2]:
imly

<module 'mlsquare.imly' from '/home/shakkeel/Desktop/mlsquare-core/mlsquare/src/mlsquare/imly/__init__.py'>

In [3]:
from sklearn.linear_model import LinearRegression
from mlsquare.imly import dope

model = LinearRegression()

m = dope(model)


Transpiling your model to it's Deep Neural Network equivalent


Using TensorFlow backend.
/home/shakkeel/anaconda3/envs/test_imly/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/home/shakkeel/anaconda3/envs/test_imly/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/shakkeel/anaconda3/envs/test_imly/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/home/shakkeel/anaconda3/envs/test_imly/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
2019-06-17 14:58:44,308	INFO node.py:423 -- Process STDOUT and STDERR is being redirected to /tmp/ray/sess

In [4]:
m

## LasLabs python plugin tutorial

### Mount point

In [44]:
import os
import imp

class PlugPyMount(type):
    ''' This class acts as a mount point for our plugins    '''

    # Default path to search for plugins - change with register_plugin_dir
    # plugin_path = os.path.join(__file__, 'plugins')
    plugin_path = os.path.join(os.path.abspath(''), 'plugins')

    def __init__(self, name, bases, attrs):
        ''' Initializing mount, or registering a plugin?    '''
        if not hasattr(self, 'plugins'):
            self.plugins = PlugPyStruct(PlugPyMount)
        else:
            self.register_plugin(self)
    
    def register_plugin(self, plugin):
        ''' Registration logic + append to plugins struct '''
        plugin = plugin() #< Init the plugin
        self.plugins[plugin.__class__.__name__] = plugin
    
    @staticmethod
    def register_plugin_dir(plugin_path):
        ''' This function sets the plugin path to be searched   '''
        if os.path.isdir(plugin_path):
            PlugPyMount.plugin_path = plugin_path
        else:
            raise EnvironmentError('%s is not a directory' % plugin_path)
        
    @staticmethod
    def find_plugins():
        ''' Traverse registered plugin directory and import non-loaded modules  '''
        plugin_path = PlugPyMount.plugin_path
        if not os.path.isdir(plugin_path):
            raise EnvironmentError('%s is not a directory' % plugin_path)
        
        for file_ in os.listdir(plugin_path):
            if file_.endswith('.py') and file_ != '__init__.py':
                module = file_[:-3] #< Strip extension
                mod_obj = globals().get(module)
                if mod_obj is None:
                    f, filename, desc = imp.find_module(
                        module, [plugin_path])
                    globals()[module] = mod_obj = imp.load_module(
                        module, f, filename, desc)

### Declare mount

In [57]:
class PlugPy(metaclass = PlugPyMount):
    ''' Default PlugPy implementation, metaclasses PlugPyMount  '''
#     __metaclass__ = PlugPyMount

### Finding a mount

In [46]:
class PlugPyStruct(dict):
    '''
        Subclass dict, re-implement __getitem__ to scan for plugins
        if a requested key is missing
    '''
    def __init__(self, cls, *args, **kwargs):
        '''
            Init, set mount to PlugPyMount master instance
            @param  PlugPyMount cls
        '''
        self.mount = cls
        super(PlugPyStruct, self).__init__(*args,**kwargs)

    def __getitem__(self, key, retry=True, default=False):
        ''' Re-implement __getitem__ to scan for plugins if key missing  '''
        try:
            return super(PlugPyStruct, self).__getitem__(key)
        except KeyError:
            if default != False:
                return default
            elif retry:
                self.mount.find_plugins()
                return self.__getitem__(key, False)
            else:
                raise KeyError(
                    'Plugin "%s" not found in plugin_dir "%s"' % (
                        key, self.mount.plugin_path
                    )
                )

### ???

In [ ]:
    def set_iter_refresh(self, refresh=True):
        '''
            Toggle flag to search for new plugins before iteration
            @param  bool    refresh     Whether to refresh before iteration
        '''
        self.iter_refresh = refresh

    def __iter__(self):
        ''' Reimplement __iter__ to allow for optional plugin refresh   '''
        if self.iter_refresh:   self.mount.find_plugins()
        return super(PlugPyStruct, self).__iter__()
    
    def values(self):
        ''' Reimplement values to allow for optional plugin refresh   '''
        if self.iter_refresh:   self.mount.find_plugins()
        return super(PlugPyStruct, self).values()
    
    def keys(self):
        ''' Reimplement keys to allow for optional plugin refresh   '''
        if self.iter_refresh:   self.mount.find_plugins()
        return super(PlugPyStruct, self).keys()
    
    def items(self):
        ''' Reimplement items to allow for optional plugin refresh   '''
        if self.iter_refresh:   self.mount.find_plugins()
        return super(PlugPyStruct, self).items()
    
    def itervalues(self):
        ''' Reimplement itervalues to allow for optional plugin refresh   '''
        if self.iter_refresh:   self.mount.find_plugins()
        return super(PlugPyStruct, self).itervalues()
    
    def iterkeys(self):
        ''' Reimplement iterkeys to allow for optional plugin refresh   '''
        if self.iter_refresh:   self.mount.find_plugins()
        return super(PlugPyStruct, self).iterkeys()
    
    def iteritems(self):
        ''' Reimplement iteritems to allow for optional plugin refresh   '''
        if self.iter_refresh:   self.mount.find_plugins()
        return super(PlugPyStruct, self).iteritems()

### Making a plugin

In [48]:
class TestPlugin(PlugPy):

    initialized = False

    def __init__(self):
        print('Initializing TestPlugin')
        self.initialized = True

    def run(self):
        print('Running TestPlugin')
        return True

### Using the plugin

In [85]:
PlugPy.register_plugin(TestPlugin) # How will "path_to_plugin_work"
PlugPy.plugins['TestPlugin'].run()

Initializing TestPlugin
Running TestPlugin


True

In [65]:
import inspect
inspect.getmodule(TestPlugin.__class__)

<module 'builtins' (built-in)>

In [71]:
PlugPy.find_plugins()

OSError: /home/shakkeel/Desktop/mlsquare-playground/cook-imly/Notebooks/plugins is not a directory

In [52]:
os.path.abspath('')

'/home/shakkeel/Desktop/mlsquare-playground/cook-imly/Notebooks'

## Nevergrad trials

In [1]:
import nevergrad as ng

In [2]:
ng

<module 'nevergrad' from '/home/shakkeel/Desktop/github_repos/nevergrad/nevergrad/__init__.py'>

In [2]:
import nevergrad as ng

def square(x, y=12):
    return sum((x - .5)**2) + abs(y)

optimizer = ng.optimizers.OnePlusOne(instrumentation=2, budget=100)
# alternatively, you could use ng.optimizers.registry["OnePlusOne"]
# (registry is a dict containing all optimizer classes)
recommendation = optimizer.minimize(square)
print(recommendation)

Candidate(args=(array([0.49429745, 0.50480782]),), kwargs={})


In [5]:
ng.optimizers.registry["OnePlusOne"](instrumentation=2, budget=100)

Instance of OnePlusOne(instrumentation=A(2), budget=100, num_workers=1)

In [10]:
ng.optimizers.registry.register('MyOptimizer')

'MyOptimizer'

In [7]:
ng.optimizers.registry.unregister('OnePlusOne')

In [13]:
ng.optimizers.registry.get_info('MyOptimizer')

ValueError: "MyOptimizer" is not registered.

TO TRY  
1) Place debuggers and observe OnePlusOne behaviour  
2) Try contributing a dummy optimizer  
3) Use registry and get a feel of it  

---

## Notes/Qs

1) Responsibilities of PlugPyMount  
2) What is PlugPy for?

---

## Tests

#### Metaclasses in python

In [93]:
class Meta(type):
    def __new__(cls, name, bases, dct):
        x = super().__new__(cls, name, bases, dct)
        x.attr = 100
        return x
    def __init__(cls, name, bases, dct):
        print('From Meta')

In [94]:
class Bar(metaclass=Meta):
    pass

# Foo.attr

From Meta


In [95]:
class test_bar(Bar):
    pass

From Meta


In [81]:
class Struct(dict):
    pass

TypeError: dict expected at most 1 arguments, got 3

In [79]:
Struct['iterate']

TypeError: 'type' object is not subscriptable

In [40]:
Meta.__file__

AttributeError: type object 'Meta' has no attribute '__file__'

In [26]:
class Plugin(object):
    class __metaclass__(type):
        def __init__(cls, name, bases, dict):
            type.__init__(name, bases, dict)
            registry.append((name, cls))

In [27]:
Plugin

__main__.Plugin

In [34]:
test=dict({'a':1})

In [37]:
test.__getitem__('a')

1

In [38]:
__file__

NameError: name '__file__' is not defined

In [42]:
import os
print(__file__)
# print os.path.join(os.path.dirname(__file__), '..')
# print os.path.dirname(os.path.realpath(__file__))
# print os.path.abspath(os.path.dirname(__file__))

NameError: name '__file__' is not defined

In [43]:
os.path.abspath('')

'/home/shakkeel/Desktop/mlsquare-playground/cook-imly/Notebooks'